In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import regularizers
import tensorflow as tf
from imblearn.over_sampling import RandomOverSampler

# Load the dataset
data = pd.read_excel('android_traffic.xlsx')
# Separate the features and the target variable
X = data.drop(['type', 'name'], axis=1)
y = data['type']

# Encode the categorical target variable
le = LabelEncoder()
y = le.fit_transform(y)


# Standardize the numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Perform random oversampling
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

# Define the model
model = Sequential()
model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))  # Adjust dropout rate

model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))  # Adjust dropout rate

model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))  # Adjust dropout rate

model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)  # Increase number of epochs

# Evaluate the model
y_pred = model.predict(X_test)
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')


Epoch 1/50
197/197 [==============================] - 5s 7ms/step - loss: 1.3354 - accuracy: 0.5843
Epoch 2/50
197/197 [==============================] - 1s 7ms/step - loss: 0.7788 - accuracy: 0.6123
Epoch 3/50
197/197 [==============================] - 1s 6ms/step - loss: 0.6883 - accuracy: 0.6463
Epoch 4/50
197/197 [==============================] - 1s 6ms/step - loss: 0.6672 - accuracy: 0.6437
Epoch 5/50
197/197 [==============================] - 1s 6ms/step - loss: 0.6605 - accuracy: 0.6605
Epoch 6/50
197/197 [==============================] - 1s 7ms/step - loss: 0.6560 - accuracy: 0.6630
Epoch 7/50
197/197 [==============================] - 1s 7ms/step - loss: 0.6540 - accuracy: 0.6557
Epoch 8/50
197/197 [==============================] - 1s 7ms/step - loss: 0.6553 - accuracy: 0.6643
Epoch 9/50
197/197 [==============================] - 1s 6ms/step - loss: 0.6525 - accuracy: 0.6654
Epoch 10/50
197/197 [==============================] - 2s 8ms/step - loss: 0.6500 - accuracy: 0.6786

In [2]:
# Save the model
model.save('android_traffic_model.h5')

In [5]:
from keras.models import load_model
# Function to input features and return predicted class labels
def predict_traffic_class(features):
    # Load the saved model
    loaded_model = load_model('android_traffic_model.h5')

    # Preprocess the input features
    features_scaled = scaler.transform(features)

    # Make predictions
    predictions = loaded_model.predict(features_scaled)
    predicted_classes = np.round(predictions).astype(int)

    # Inverse transform the predicted classes to get the original labels
    predicted_labels = le.inverse_transform(predicted_classes)

    return predicted_labels

# Example usage of the predict_traffic_class function
input_features = np.array([[100, 0.5, 3, 5000, 20, 1, 200, 150, 8000, 6000, 10, 10]])
predicted_class = predict_traffic_class(input_features)
print(f'Predicted class: {predicted_class[0]}')

c:\Users\z035793\Documents\others\side_projects\malicious-network\env\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 300ms/step
Predicted class: benign


c:\Users\z035793\Documents\others\side_projects\malicious-network\env\lib\site-packages\sklearn\preprocessing\_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
